In [2]:
#@title Librerias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import sympy as sp

# Actividad 03: Errores y estabilidad

---
### Profesor: Juan Marcos Marín
### Nombre: Isaac Villada Montoya
*Métodos computacionales 2024-II*

---

#1. Serie Seno
Considere la serie de Taylor para el seno

$$\sin(x) = \sum^{\infty}_{n=1} (-1)^{n}\frac{x^{2n+1}}{(2n+1)!} $$

a. Defina una función que a partir de la *serie de Taylor* y un valor de $x$ en radianes encuentre el valor de $\sin(x)$.


b. Usando $x$ como `float32` encuentre cuántos términos de la serie son necesarios para obtener una convergencia con un error absoluto menor al $ϵ$ de la máquina para dicho tipo de variable. Para el valor exacto use `math.sin()`.


c. Ahora, usando $x$ como `float64` encuentre cuántos términos de la serie son necesarios para obtener una convergencia con un error absoluto menor al $ϵ$ de la máquina para dicho tipo de variable. Para el valor exacto use `math.sin()`.


d. Describa los errores porcentuales para cada cálculo y por qué pueden existir diferencias.

In [3]:
import math

def taylor_senoi(angulo, tol) :

  seno = 0
  real = math.sin(angulo)
  i = 0

  while abs(real - seno) > tol :

    seno += (-1)**i * angulo**(2*i + 1)/math.factorial(2*i + 1)
    i += 1

  return i, seno


def taylor_seno(angulo, tol) :

  seno = 0
  real = math.sin(angulo)
  i = 0

  while abs(real - seno) > tol :

    seno += (-1)**i * angulo**(2*i + 1)/math.factorial(2*i + 1)
    i += 1

  return seno

taylor_senoi(math.pi/2, 0.00001)

(5, 1.0000035425842861)

In [4]:
import numpy as np

x = math.radians(80)
tol_ = np.finfo(np.float32).eps
N, seno = taylor_senoi(x, tol_)


print(f"terminos: {N}")

terminos: 6


In [5]:
x = math.radians(80)
tol_ = np.finfo(np.float64).eps
N, seno = taylor_senoi(x, tol_)


print(f"terminos: {N}")

terminos: 10


In [6]:
def error_porcentual(exact, aprox) :

  error_porcentual = (abs(exact-aprox)/abs(exact)) * 100
  return error_porcentual

tol_ = np.finfo(np.float32).eps


print('El error porcentual entre el valor real mediante la función math.sin y la función taylor_seno32 es :',
      error_porcentual(math.sin(math.pi/2), taylor_seno(math.pi/2, tol_)), '%')

tol_ = np.finfo(np.float64).eps

print('El error porcentual entre el valor real mediante la función math.sin y la función taylor_seno64 es :',
      error_porcentual(math.sin(math.pi/2), taylor_seno(math.pi/2, tol_)), '%')


El error porcentual entre el valor real mediante la función math.sin y la función taylor_seno32 es : 5.625894905492146e-06 %
El error porcentual entre el valor real mediante la función math.sin y la función taylor_seno64 es : 0.0 %


#2. Serie Exponencial
Considere la serie para $e^{-x}$

\begin{equation}
e^{-x}=\sum_{n=0}^{N}  (-1)^n \frac{x^{n}}{n!}
\end{equation}

a. Calcula la serie para $x \le 1$ y compárela con la función incorporada `np.exp(x)` (asuma que la función exponencial incorporada es exacta). Elegir un $N$ para el cual el siguiente término en la serie no sea más que $10^{-7}$ de la suma hasta ese punto.

\begin{equation}
\left| \frac{(-x)^{N+1}}{(N+1)!} \right | \le \left| 10^{-7} \sum_{N=0}^{N} \frac{(-x)^{n}}{n!} \right|
\end{equation}


b. Examine los términos de la serie para $x\approx 10$ y observa las cancelaciones sustractivas significativas que ocurren cuando términos grandes se suman para dar respuestas pequeñas. En particular, imprime la cancelación casi perfecta en $n \approx x − 1$.

c. Compruebe si se obtiene una mejor precisión siendo ingenioso y usando $e^{−x} = \frac{1}{e^x}$ para valores grandes de $x$. Esto elimina la cancelación sustractiva, pero no elimina todos los errores de redondeo.


d. Incrementando progresivamente $x$ de 1 a 10, y luego de 10 a 100, usa el programa para determinar experimentalmente cuándo la serie comienza a perder precisión, y cuándo la serie ya no converge.


e. Realiza una serie de gráficos del error versus $N$ para diferentes valores de $x$.

In [7]:
import math
import numpy as np

def taylor_e(x, tol) :

  ex = 1
  ex_term = 1
  real = np.exp(-x)
  i = 1

  while abs(real - ex) > tol :

    ex_term= (-1)**i * (x**i)/math.factorial(i)
    ex += ex_term
    i += 1

  return i, ex

taylor_e(0.1, 0.01)

(2, 0.9)

In [8]:
def taylor_en(x,N) :

  ex = 1
  ex_term = 1
  real = np.exp(-x)
  i = 1

  for i in range(N) :

    ex_term = (-1)**i * (x**i)/math.factorial(i)
    ex += ex_term
    i += 1

    if abs(ex_term + 1) <= abs(10e-7 * ex) :

      break

  return i, ex

taylor_en(0.5, 100)

(100, 1.6065306597126336)

In [9]:
def taylore_term(x, tol) :

  ex = 1
  ex_term = 1
  real = np.exp(-x)
  i = 1

  while abs(real - ex) > tol :

    ex_term= (-1)**i * (x**i)/math.factorial(i)
    ex += ex_term
    i += 1

  return ex_term

taylore_term(0.1, 0.01)

-0.1

# 3. Serie $S_N$

Un ejemplo de la cancelacion substractiva es en la suma de series alternantes. Considere la serie $\mathrm{S}_N$,  escrita  matematicamente de tres formas diferentes:

-
\begin{equation}
\mathrm{S}_N^{(1)} = \sum_{n = 1}^{2N} (-1)^n \frac{n}{n+1}
\end{equation}

- Sumando números pares e impares.

\begin{equation}
\mathrm{S}_N^{(2)} = -\sum_{n = 1}^{N}\frac{2n-1}{2n} + \sum_{n=1}^{N} \frac{2n}{2n+1}
\end{equation}

- Combinando las series de una forma analítica

\begin{equation}
\mathrm{S}_N^{(3)} = \sum_{n = 1}^{N}\frac{1}{2n(2n+1)}
\end{equation}



Las tres sumas $\mathrm{S}_N^{(1)}$, $\mathrm{S}_N^{(2)}$ y $\mathrm{S}_N^{(3)}$ son matemáticamente iguales, pero los resultados númericos  pueden tener diferencias.


a.  Escribir tres funciones para que realicen el cálculo de $\mathrm{S}_N^{(1)}$, $\mathrm{S}_N^{(2)}$ y $\mathrm{S}_N^{(3)}$


b. Estudiar la convergencia de la serie para los tres casos.


c. Considere  que $S_N^{(3)}$ es la solución exacta, realizar dos graficos log-log, del error relativo en función
del número de términos ($\mathrm{Log_{10}} E$ VS $\mathrm{Log_{10}} N$), para $S_N^{(1)}$ y $S_N^{(2)}$. Donde,  $$E = |(S_N^{(i)} - S_N^{(3)})/S_N^{(3)}|,$$ $i = 1, 2$. Considere valores de $N = 1\times 10^{6}$.

d. Analizar y discutir los resultados. Tenga en cuenta la discusión realizada sobre teoría de errores en la página
39 del libro [Computational Physics. ](https://www.dropbox.com/s/n06ul2r4l65khl6/Computational%20Physics%20-%20Problem%20Solving%20with%20Computers%2C%203527406263.pdf?dl=0)

In [12]:
def s1(N) :

  s = 0

  for i in range(1, 2*N) :

    s += (-1)**i * i/(i+1)
    i += 1

  return s

def s2(N) :

  s = 0

  for i in range(1, 2*N) :

    s += (-1) * ((2*i-1)/(2*i)) + ((2*i)/(2*i+1))
    i += 1

  return s

def s3(N) :

  s = 0

  for i in range(1, N) :

    s += 1/(2*i)*(2*i+1)
    i += 1

  return s